# Performing a chemical equilibrium calculation

This tutorial demonstrates how to use Reaktoro to perform a chemical equilibrium calculation. We start by importing the `reaktoro` package:

In [7]:
from reaktoro import *

Next, we need a thermodynamic database that enable us to compute thermodynamic properties of species and reactions. For this, we create an object of class [Database]:

In [8]:
db = Database("supcrt98.xml")

To indicate the phases of interest (as well as their species) that may potentially exist at equilibrium, we create an object of class [ChemicalEditor]:

In [9]:
editor = ChemicalEditor(db)

We consider an aqueous phase composed of all aqueous species in the database that can be formed with the given chemical elements below:

In [10]:
editor.addAqueousPhaseWithElements("H O Na Cl C Ca Mg Si")

> **Note:** This automatic selection of chemical species for a phase can result in a large number of them. This potentially increases the computing cost of the chemical reaction calculations. If you are using Reaktoro in demanding applications, you might want to manually specify the chemical species of each phase in your chemical system. This can be achieved by providing an explicit list of species names, e.g., `editor.addAqueousPhase("H2O(l) H+ OH- CO2(aq)")`. Note, however, that care is required here to ensure relevant species are not missing. The just given example is a bad one in fact, with important species such as `HCO3-` and `CO3--` missing in the list!.

We are interested in a gaseous phase containing exactly the following gases (which may or not exist in positive amounts at the end of our equilibrium calculation later!):

In [11]:
editor.addGaseousPhase("H2O(g) CO2(g)")

Finally, we consider some pure minerals that could exist in positive amounts in our equilibrium calculations:

In [12]:
editor.addMineralPhase("Halite")
editor.addMineralPhase("Calcite")
editor.addMineralPhase("Magnesite")
editor.addMineralPhase("Dolomite")
editor.addMineralPhase("Quartz")

Next follows an important step, where we create the chemical system with the information so far collected in the [ChemicalEditor] object `editor`:

In [13]:
system = ChemicalSystem(editor)

We use class [EquilibriumProblem] to specify the conditions at which our system should be in equilibrium.

In [14]:
problem = EquilibriumProblem(system)

In particular, we can specify temperature and pressure as well as the initial condition for substance amounts

In [15]:
problem.setTemperature(70, "celsius")
problem.setPressure(100, "bar")
problem.add("H2O", 1.0, "kg")
problem.add("CO2", 2.0, "mol")
problem.add("NaCl", 1.0, "mol")
problem.add("CaCO3", 10.0, "g")
problem.add("MgCO3", 5.0, "g")
problem.add("Quartz", 1.0, "mol")

> **Note:** The substance names above can either be chemical formulas, such as CaCO3 and CaCl2, as well as names of species that can be found in the database, such as Quartz. Reaktoro will break down the chemical formulas of the substances and calculate the amount of each chemical element in the system. These element amounts are inputs to the equilibrium calculation. In the future, we will only allow species names to be provided, since this is a safer way of preventing unfeasible elemental mass conditions to be imposed (e.g., there are *x* moles of C and *y* moles of O, and distributing these among the species always produce an excess of either C or O).

We now perform a fast Gibbs energy minimization calculation to compute the chemical equilibrium state of the system at given conditions stored in `problem`. For this, we use the convenient function `equilibrate`:

In [17]:
state = equilibrate(problem)

## Using class ChemicalState to inspect species amounts

The result of the `equilibrate` call before, `state`, is an object of class [ChemicalState]. This object contains the temperature, pressure, and amounts of the species at the computed chemical equilibrium state. We can access these properties as follows:

In [ ]:
T = state.temperature()
P = state.pressure()
n = state.speciesAmounts()

In [ ]:
print(f"T = {T} K")
print(f"P = {P} Pa")
print(f"n = (in mol)\n{n}")

To print the name of each species and its amount (in mol), we execute the following loop:

In [ ]:
for species in system.species():
    name = species.name()
    amount = state.speciesAmount(name)
    print(f"{name:>15} = {amount}")

You can also output the chemical state to a file

In [ ]:
state.output("results/state.txt")

## Using the class ChemicalProperties to obtain species activities

If you require chemical properties of a system that depend on temperature (*T*), pressure (*P*), and composition (*n*), then [ChemicalProperties] class is what you need

In [21]:
properties = ChemicalProperties(system)

We can compute the chemical properties of the system at the state of equilibrium we found before:

In [ ]:
properties.update(T, P, n)

Alternatively, we could also have done:

In [23]:
properties = state.properties()

> **Note:** The call above creates a new object of [ChemicalProperties] each time. If you are using Reaktoro in a simulator that needs the chemical properties of the system at millions/billions of states each time step, prefer to the `update` method of an existing [ChemicalProperties] object.

Once we have computed the chemical properties, we can query for some of them. Below we get the natural log of the species activities:

In [24]:
lna = properties.lnActivities().val

> **Note:** The use of `.ddT`, `.ddP`, and `.ddn`, instead of `.val`, extracts the derivatives of the activities (or any other chemical property) with respect to *T*, *P*, and *n*, respectively.

To compute the actual activities (not their natural log), and print them one by one, we do

In [ ]:
a = numpy.exp(lna)
for i, species in enumerate(system.species()):
    print(f"{species.name():>15} = {a[i]}")

## Calculating the pH of the aqueous solution

Let's create a pH function that computes the pH of the aqueous solution given the chemical properties of the system (this will be soon simplified!)

In [27]:
evaluate_pH = ChemicalProperty.pH(system)
pH = evaluate_pH(properties)
print(f"The pH of the aqueous phase is {pH.val}.")
print(f"Its sensitivity with respect to speciation, ∂(pH)/∂n, is:")
for i, species in enumerate(system.species()):
    print(f"{species.name():>15} = {pH.ddn[i]}")

The pH of the aqueous phase is 4.982898186792658.
Its sensitivity with respect to speciation, ∂(pH)/∂n, is:
         CO(aq) = 0.007463272238343973
        CO2(aq) = 0.007463272238343973
          CO3-- = 0.01764093171816751
      Ca(HCO3)+ = 0.010007687108299856
     Ca(HSiO3)+ = 0.010007687108299856
           Ca++ = 0.01764093171816751
      CaCO3(aq) = 0.007463272238343973
          CaCl+ = 0.010007687108299856
      CaCl2(aq) = 0.007463272238343973
          CaOH+ = 0.010007687108299856
            Cl- = 0.010007687108299856
           ClO- = 0.010007687108299856
          ClO2- = 0.010007687108299856
          ClO3- = 0.010007687108299856
          ClO4- = 0.010007687108299856
             H+ = -26542.249160597315
         H2(aq) = 0.007463272238343973
         H2O(l) = 0.007370231336413049
       H2O2(aq) = 0.007463272238343973
          HCO3- = 0.010007687108299856
        HCl(aq) = 0.007463272238343973
       HClO(aq) = 0.007463272238343973
      HClO2(aq) = 0.00746327223834397

[ChemicalEditor]: https://reaktoro.org/cpp/classReaktoro_1_1ChemicalEditor.html
[ChemicalProperties]: https://reaktoro.org/cpp/classReaktoro_1_1ChemicalProperties.html
[ChemicalState]: https://reaktoro.org/cpp/classReaktoro_1_1ChemicalState.html
[ChemicalSystem]: https://reaktoro.org/cpp/classReaktoro_1_1ChemicalSystem.html
[Database]: https://reaktoro.org/cpp/classReaktoro_1_1Database.html
[EquilibriumProblem]: https://reaktoro.org/cpp/classReaktoro_1_1EquilibriumProblem.html